In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import json
import pickle
from transformers import BertTokenizer
import torch
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm

In [2]:
# data_path = os.environ['DATA_PATH']
data_path = "D:/Data/neural-punctuator/ted_dataset/"
os.listdir(data_path)

['.DS_Store',
 'COPYRIGHT',
 'data.pkl',
 'encoded.pkl',
 'fix_metadata.py~',
 'raw_test_texts.txt',
 'raw_train_texts.txt',
 'raw_valid_texts.txt',
 'README',
 'stats.py',
 'stats.py~',
 'szeged',
 'szeged.allChunks.newKr.unshuffled',
 'targets.pkl',
 'ted_talks-10-Sep-2012.json',
 'ted_talks-25-Apr-2012.json',
 'ted_users-10-Sep-2012.json',
 'ted_users-25-Apr-2012.json',
 'test_data.pkl',
 'text.pkl',
 'texts.pkl',
 'train_data.pkl',
 'valid_data.pkl']

In [3]:
file_path = data_path + "ted_talks-25-Apr-2012.json"

In [4]:
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [5]:
transcripts = [d['transcript'] for d in data]

In [6]:
len(transcripts)

1149

In [7]:
transcripts = list(set(transcripts))
len(transcripts)

1069

In [8]:
transcripts = [t.replace('\n', ' ') for t in transcripts if len(t) > 0]
len(transcripts)

1068

In [9]:
raw_text = '\n'.join(transcripts)
len(raw_text.split(' '))

2464687

In [10]:
def clean_text(text):
    escape_words = (' (Laughter)', ' (Applause)', ' (Music)', ' (Cheering)', ' (Singing)', ' (Video)', '')

    for ew in escape_words:
        text = text.replace(ew, '')

    text = text.replace('!', '.')
    text = text.replace(':', ',')
    text = text.replace('--', ',')
    text = text.replace('-', ',')
    text = text.replace(' ,', ',')
    text = text.replace('♫', '')

    text = re.sub(r'--\s?--', '', text)
    text = re.sub(r'\s+', ' ', text)

    text = text.replace(' ,', ',')
    
    return text

In [11]:
ts = []
for t in tqdm(transcripts):
    ts.append(clean_text(t))
transcripts = ts

In [12]:
print(transcripts[3])

Imagine spending seven years at MIT and research laboratories, only to find out that you're a performance artist. I'm also a software engineer, and I make lots of different kinds of art with the computer. And I think the main thing that I'm interested in is trying to find a way of making the computer into a personal mode of expression. And many of you out there are the heads of Macromedia and Microsoft, and in a way those are my bane, I think there's a great homogenizing force that software imposes on people and limits the way they think about what's possible on the computer. Of course, it's also a great liberating force that makes possible, you know, publishing and so forth, and standards, and so on. But, in a way, the computer makes possible much more than what most people think, and my art has just been about trying to find a personal way of using the computer, and so I end up writing software to do that. Chris has asked me to do a short performance, and so I'm going to take just th

In [13]:
albert_pretrained = torch.hub.load('huggingface/pytorch-transformers', 'model', 'albert-base-v1')
albert_tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'albert-base-v1')
albert_pretrained.eval()

Using cache found in C:\Users\gbenc/.cache\torch\hub\huggingface_pytorch-transformers_master
Using cache found in C:\Users\gbenc/.cache\torch\hub\huggingface_pytorch-transformers_master


AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(30000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0.1, inplace=False)
  

In [14]:
# %%timeit
# albert_tokenizer.tokenize(" , ! ? . ... .. asd. qwe.. tyu, uio , asdfs")

In [15]:
# %%timeit
# albert_tokenizer.encode(" , ! ? . ... .. asd. qwe.. tyu, uio , asdfs")

In [16]:
target_ids = (9, 60, 15)
albert_tokenizer.encode(".?,")

[2, 13, 9, 60, 15, 3]

In [17]:
encoded_texts = []
for t in tqdm(raw_text.split('\n')):
    encoded_texts.append(albert_tokenizer.encode(t))

with open(data_path + 'encoded.pkl', 'wb') as f:
    pickle.dump(encoded_texts, f)

Token indices sequence length is longer than the specified maximum sequence length for this model (3611 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1586 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5258 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (546 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1400 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (3506 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2077 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1177 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4135 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4213 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Token indices sequence length is longer than the specified maximum sequence length for this model (4259 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (655 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3287 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4138 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3128 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (706 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2068 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1193 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4095 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3922 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (3561 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5059 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3911 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4571 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4729 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

...

Token indices sequence length is longer than the specified maximum sequence length for this model (1665 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3614 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3513 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5571 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3877 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Token indices sequence length is longer than the specified maximum sequence length for this model (2401 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4498 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3624 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1975 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3313 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Token indices sequence length is longer than the specified maximum sequence length for this model (3816 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2726 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3607 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1407 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (952 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (3533 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2802 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2589 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2581 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4138 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Token indices sequence length is longer than the specified maximum sequence length for this model (4273 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4097 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2274 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4072 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4599 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

In [18]:
# with open(data_path + 'encoded.pkl', 'rb') as f:
#     encoded_texts = pickle.load(f)

In [19]:
id2target = {-1: 0,
              9: 1, # .
              60: 2, # ?
              15: 3,  # ,
              -2: -1, # will be masked
             }
target2id = {value: key for key, value in id2target.items()}
    
    
def create_target(encoded):
    targets = []
    text = []

    target = -2 # Always mask after [CLS] token

    text.append(encoded[0])
    for word in encoded[1:]:
        if word in id2target.keys():
            target = word
        else:
            if not albert_tokenizer._convert_id_to_token(word).startswith('▁'):
                target = -2
            targets.append(target)
#             print(target)
#             print(albert_tokenizer._convert_id_to_token(word), '\t', end="")
            target = -1
            text.append(word)

    targets.append(target)

    targets = [id2target[t] for t in targets]

    return text, targets

In [20]:
texts = []
targets = []
for text in tqdm(encoded_texts):
    text, target = create_target(text)
    texts.append(text)
    targets.append(target)

In [21]:
n = -120
test_n = -4#int(.96*len(texts))
train_texts = texts[:n]
train_targets = targets[:n]
valid_texts = texts[n:test_n]
valid_targets = targets[n:test_n]
test_texts = texts[test_n:]
test_targets = targets[test_n:]
len(train_texts), len(valid_texts), len(test_texts)

(948, 116, 4)

In [22]:
len(' '.join(raw_text.split('\n')[:n]).split(' ')), \
len(' '.join(raw_text.split('\n')[n:test_n]).split(' ')), \
len(' '.join(raw_text.split('\n')[test_n:]).split(' '))

(2183846, 266634, 15274)

In [23]:
with open(data_path + 'raw_train_texts.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(raw_text.split('\n')[:n]) + "\n")
with open(data_path + 'raw_valid_texts.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(raw_text.split('\n')[n:test_n]) + "\n")
with open(data_path + 'raw_test_texts.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(raw_text.split('\n')[test_n:]) + "\n")

In [24]:
np.sum(np.array(valid_targets[0]) != -1)

2981

In [25]:
sum((np.array(vt) != -1).sum() for vt in valid_targets)

262960

In [26]:
with open(data_path + 'train_data.pkl', 'wb') as f:
    pickle.dump((train_texts, train_targets), f)
with open(data_path + 'valid_data.pkl', 'wb') as f:
    pickle.dump((valid_texts, valid_targets), f)
with open(data_path + 'test_data.pkl', 'wb') as f:
    pickle.dump((test_texts, test_targets), f)

# with open(data_path + 'train_data.pkl', 'rb') as f:
#     train_texts, train_targets = pickle.load(f)

In [27]:
from collections import Counter
c = Counter((t for targets in train_targets for t in targets))

In [28]:
def get_weights_for_sampler(train_data):
    # used because of the imbalanced dataset
    train_data['sector_int'], _ = pd.factorize(train_data['sector'])

    class_sample_count = np.array(
        [len(np.where(train_data['sector_int'] == t)[0]) for t in np.unique(train_data['sector_int'])])
    weight = 1. / class_sample_count
    samples_weight = np.array([weight[t] for t in train_data['sector_int']])

    samples_weight = torch.from_numpy(samples_weight)
    samples_weig

In [29]:
targets = np.array([t for targets in train_targets for t in targets])
targets.shape, targets[:2]

((2526522,), array([-1, -1]))

In [30]:
output_dim = 4

weights = np.zeros((output_dim, ))
for t in range(output_dim):
    count = (targets == t).sum()
    weights[t] = count

weights /= weights.sum()
weights

array([0.87099139, 0.05776514, 0.00432066, 0.06692281])

In [31]:
c

Counter({-1: 357411, 0: 1889277, 1: 125299, 3: 145163, 2: 9372})

In [32]:
# .?,
s = 0
weights = []
for key in range(output_dim):
    value = c[key]
    s += value
for key in range(output_dim):
    value = c[key]
    print(key, target2id[key], albert_tokenizer.decode(target2id[key]) if key > 0 else "\"\"", value, value/s)
    weights.append(s/value)

0 -1 "" 1889277 0.8709913877159814
1 9 . 125299 0.057765139727750214
2 60 ? 9372 0.00432066408772995
3 15 , 145163 0.0669228084685385


In [33]:
import numpy as np
weights = np.array(weights)
weights /= weights.sum()
weights

array([0.004335  , 0.06536381, 0.87388172, 0.05641947])